In [6]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier

df = pd.read_csv('../data/GeneralDatensatz18-21ohneGeo-mitLockdown_mitCorona.csv', sep=';')

X=df[['UMONAT','USTUNDE','UWOCHENTAG','UART','USTRZUSTAND','BEZ','UTYP1','ULICHTVERH','IstRad','IstPKW','IstFuss','IstKrad','IstGkfz','IstSonstige', 'LOCKDOWN', 'COVID']]

#für tödliche und schwere vs. leichte Unfälle
y = df['UKATEGORIE'].isin([1, 2]).astype(int)
# -> 1 ist schwer oder tödlich, 0 ist leicht

# KFold-Konfiguration
kf = KFold(n_splits=5, shuffle=True, random_state=42)

import matplotlib.pyplot as plt

# Leere Listen für F1-Scores
f1_scores_log_reg = []
f1_scores_rf = []

weights = range(0, 1500, 25)

for i in weights:
    # Logistische Regression
    log_reg = LogisticRegression(max_iter=1000, class_weight={0:1, 1: i})

    # F1-Score berechnen und speichern
    f1_reg = cross_val_score(log_reg, X, y, cv=kf, scoring='f1')
    f1_scores_log_reg.append(f1_reg.mean())

    # Random Forest
    rf_clas = RandomForestClassifier(n_estimators=100, max_depth=5,random_state=42, class_weight={0:1, 1: i})

    # F1-Score berechnen und speichern
    f1_rf = cross_val_score(rf_clas, X, y, cv=kf, scoring='f1')
    f1_scores_rf.append(f1_rf.mean())

# F1-Scores plotten
plt.figure(figsize=(10, 6))
plt.plot(weights, f1_scores_log_reg, marker='o', label='Logistic Regression')
plt.plot(weights, f1_scores_rf, marker='o', label='Random Forest')
plt.title('F1-Scores für verschiedene Gewichte')
plt.xlabel('Gewicht')
plt.ylabel('F1-Score')
plt.legend()
plt.grid(True)
plt.show()

KeyboardInterrupt: 